<a href="https://colab.research.google.com/github/SSPYX/Colab/blob/main/ExcelFormat_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy.core.multiarray import concatenate
import pandas as pd
import openpyxl
import numpy as np
import re
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font

def adjust_merged_cells(sheet_name):
   #print("Inside function", sheet_name)
   merged_ranges = sheet_name.merged_cells.ranges
   print("Merged_range", merged_ranges)
   for merged_range in merged_ranges:
    min_row, min_col, max_row, max_col = merged_range.min_row, merged_range.min_col, merged_range.max_row, merged_range.max_col
    print("Merged Min Col", min_col, "Merged Max Col", max_col, "Merged Min row", min_row, "Merged Max row", max_row)
    hidden_columns = []
    for col in range(min_col, max_col + 1):
        if sheet_name.column_dimensions[openpyxl.utils.get_column_letter(col)].hidden:
                hidden_columns.append(col)
    #sheet_name.unmerge_cells(start_row=min_row, start_columns=min_col, end_row=max_row, end_col=max_col)
    for col in range(max_col, min_col - 1, -1):
        if col not in hidden_columns:
          for row in range(min_row, max_row + 1):
              cell_value = sheet_name.cell(row=row, column=col).value
              if cell_value:
                      sheet_name.cell(row=row, column=min_col).value = cell_value
                      break
    sheet_name.merge_cells(start_row=min_row, start_column=min_col, end_row=max_row, end_column=min_col + len(hidden_columns))

wb = openpyxl.load_workbook('/content/DoT (Cluster) AR.xlsx', data_only=False, rich_text=True)
indx_sheet = wb['Index']
indx_sheet
add_col_idx = indx_sheet.max_column + 1
#sheet_names = wb.sheetnames
# Use the following SWITCH to define "scope" of the script. False = Do not apply, True = Apply
Font_Size_Switch = True
Row_Height_Switch = True
Border_Switch = False # False will not remove vertical borders
Column_Width_Switch = True
#print(wb.sheetnames)#
for title in wb.sheetnames:
  print("Start of", title)
  ws = wb[title]
#  hidden_flg = ws.sheet_state
  for name_range in wb.defined_names.values():
   char_idx = name_range.attr_text.find('!')
   #print(ws.sheet_state)
   if ws.title==name_range.attr_text[:char_idx] and ws.sheet_state == 'visible':
    #named_array = wb.defined_names[title]
    #print("I am processing",ws.title)
    cell_range=name_range.attr_text
    #print(cell_range)
    start_cell, end_cell = cell_range.split(':')
    sheet, start_cell = start_cell.split('!')
    start_cell = start_cell.replace("$","")
    end_cell = end_cell.replace("$","")
    start_row, start_col = openpyxl.utils.cell.coordinate_to_tuple(start_cell)
    end_row, end_col= openpyxl.utils.cell.coordinate_to_tuple(end_cell)
    row_height = 14.5
    index = 0
    num_count = 0
    cnt_nts = 0
    hidden_columns = []
    notes_exists = False
    min=0
    max=0
    h_min=0
    h_max=0
    true_end = end_col
    hiddenList=[]
    groupedList=[]

    adjust_merged_cells(ws)
    #Thi block loops on rows defined in the named range to find Notes colum
    for row in ws.iter_rows():
        for row_cell in row:
         if row_cell.value =="Notes" or row_cell.value =="Note":
          notes_exists=True
          cnt_nts = row_cell.col_idx
          print("Notes Index position",cnt_nts)
          break
        if notes_exists:
          break
    # This block determines number of numeric columns

    for col_h in range(start_col , end_col + 1):
        sheet_cell = ws.cell(row=end_row, column=col_h)
        last_row_values = ws.cell(row=end_row, column=col_h)
        #if ws.title == 'AR_044_ROU':
          #print("sheet scope", start_col, end_col )
          #print(col_h, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden, ws.title )
        if col_h in groupedList: #col_h == true_end and not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min:
          ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min = col_h
          ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max = col_h
          num_count += 1
          print("I am columns", col_h,"I am in condition 0")
          continue
        if col_h in hiddenList: #min and max and col_h > min and col_h <= max and not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden :
             #print("I am column", col_h, "and I am in condition 1")
             continue
        elif not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min == ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max: #This is a default scenario for visible columns
             #print(type(sheet_cell.value))
             if (re.match(r'^#',last_row_values.number_format)) and col_h != cnt_nts: #isinstance(sheet_cell.value, (int, float)): #
                  num_count += 1
                  print("I am column", col_h, "and I am in condition 2")
        elif ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min != ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max:   #This capture the first cell of hidden cell range
              hiddenList.clear()
              for i in range(ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max + 1):
                            hiddenList.append(i)
              h_min = ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min
              h_max = ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max
              #print("I am column", col_h, "and I am in condition 3", hiddenList )
              continue
        elif not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min != ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max:   #This captures any grouped columns that have min & max defined None and are visible in excel
              groupedList.clear()
              print("Condition 4 start", col_h)
              for i in range(ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max + 1):
                            groupedList.append(i)
              min = ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min
              max = ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max
              num_count += 1
              print("I am column", col_h, "and I am in condition 4",min,max, groupedList)

    print("Numeric columns", num_count, ws.title)


    #Loop on rows defined in the named range
    for row in range(start_row, end_row + 1):
      index += 1
      if Row_Height_Switch is True and index != 1:
        #set the row height to 14.5
        ws.row_dimensions[row].height=row_height
        Is_hidden = ws.row_dimensions[row].hidden
        if all(cell.value is None for cell in ws[row]):
            #print("row {} is empty".format(row))
            ws.row_dimensions[row].height=8
      groupedList.clear()
      hiddenList.clear()
      col_index=start_col + 1
# Loop on columns starting from the range defined in Named Range
      for col in range(start_col , end_col + 1):
       sheet_cell = ws.cell(row=row, column=col)
       if index==1: #Row 1
        #last_row_values = ws.cell(row=end_row, column=col)
#******This block is to hide columns that come as visible due to excel functionality to Group columns that are hidden consecutively hidden********
        if col in groupedList: #col_h == true_end and not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min:
          fstyle_bold = sheet_cell.font.bold
          fstyle_italic = sheet_cell.font.italic
        if col in hiddenList: #min and max and col_h > min and col_h <= max and not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden :
          ws.column_dimensions[openpyxl.utils.get_column_letter(col)].hidden = True
          continue
        elif not ws.column_dimensions[openpyxl.utils.get_column_letter(col)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min == ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max: #This is a default scenario for visible columns
          fstyle_bold = sheet_cell.font.bold
          fstyle_italic = sheet_cell.font.italic
        elif ws.column_dimensions[openpyxl.utils.get_column_letter(col)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min != ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max:   #This capture the first cell of hidden cell range
          hiddenList.clear()
          for i in range(ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max + 1):
                            hiddenList.append(i)
          #print("I am column", col, "and I am in condition 3", hiddenList )
          continue
        elif not ws.column_dimensions[openpyxl.utils.get_column_letter(col)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min != ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max:   #This captures any grouped columns that have min & max defined None and are visible in excel
          groupedList.clear()
          for i in range(ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max + 1):
                            groupedList.append(i)
          fstyle_bold = sheet_cell.font.bold
          fstyle_italic = sheet_cell.font.italic
          #print("I am column", col, "and I am in condition 4",min,max, groupedList)
#********End of block*************

        if not Font_Size_Switch:
            font_size = sheet_cell.font.size
        else:
            # Set the font SIZE based on numeric columns
            if num_count <= 8:
              font_size = 9
            elif num_count in (9, 10):
              font_size = 8
            else:
              font_size = 7

 #******This block sets the Column Width at row of the defined range as this block is executed within Index 1 if condition above
        if Column_Width_Switch is True:
          if notes_exists and col==col_index:
              if num_count == 1:
                   col_width = 8.91
              elif num_count == 2:
                   col_width = 8.91
              elif num_count==3:
                   col_width = 8.91
              elif num_count==4:
                   col_width=8.18
              elif num_count==5:
                   col_width=7.82
              elif num_count==6:
                   col_width=8.91
              elif num_count==7:
                   col_width=8.55
              elif num_count==8:
                   col_width=8.18
              elif num_count==9:
                   col_width=7.82
              elif num_count==10:
                   col_width=7
              elif num_count==11:
                   col_width=6.18
              else:
                   col_width=6.18
              print("I am using", col_width, "for Notes columns in sheet", ws.title)
                # now update the worksheet with above column widths
              ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=col_width
          if col==start_col and not notes_exists: #This is for column 1
                #Set the default column WIDTH of first colum based on how many numeric columns
                if num_count == 1:
                    col_width = 65.55 + 8.91
                elif num_count == 2:
                    col_width = 53.55 + 8.91
                elif num_count==3:
                    col_width = 41.45 + 8.91
                elif num_count==4:
                    col_width=34.18 + 8.18
                elif num_count==5:
                    col_width=26.09 + 7.82
                elif num_count==6:
                    col_width=49 + 8.91
                elif num_count==7:
                    col_width=40.82 + 8.55
                elif num_count==8:
                    col_width=33.64 + 8.18
                elif num_count==9:
                    col_width=27.55 + 7.82
                elif num_count==10:
                    col_width=27.91 + 7
                elif num_count==11:
                    col_width=21.64 + 6.18
                else:
                    col_width=21.64 + 6.18
                ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=col_width
                # set the column widths in worksheet
                print(col, col_width, "for label")
          if col==start_col and notes_exists: #This is for column 1
                #Set the default column WIDTH of first colum based on how many numeric columns
                if num_count == 1:
                    col_width = 65.55
                elif num_count == 2:
                    col_width = 53.55
                elif num_count==3:
                    col_width = 41.45
                elif num_count==4:
                    col_width=34.18
                elif num_count==5:
                    col_width=26.09
                elif num_count==6:
                    col_width=49
                elif num_count==7:
                    col_width=40.82
                elif num_count==8:
                    col_width=33.64
                elif num_count==9:
                    col_width=27.55
                elif num_count==10:
                    col_width=27.91
                elif num_count==11:
                    col_width=21.64
                else:
                    col_width=21.64
                # set the column widths in worksheet
                print(col, col_width, "for label")
                ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=col_width
                print(ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width)
          if (re.match(r'^#',last_row_values.number_format)) and col!=cnt_nts and col!=start_col: #s: #isinstance(last_row_values.value and col!=cnt_nts, (int, float))
                #Set the default column WIDTH for numeric column based on how many amount columns are in sheet
                if num_count <= 3 or num_count==6:
                   col_width = 11.45
                elif num_count == 4:
                   col_width = 10.36
                elif num_count==5:
                   col_width = 9.91
                elif num_count==7:
                   col_width=10.91
                elif num_count==8:
                   col_width=10.36
                elif num_count==9:
                   col_width=9.64
                elif num_count==10:
                   col_width=8.82
                else:
                   col_width=7.91
                print("I am using", col_width, "for amount")
                # set the column widths in worksheet
                ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=col_width
        elif Column_Width_Switch is False:
          ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width
#********This block sets the font size and formatting properties of cell***************
       font_prpts = {}
       if Font_Size_Switch and index == 1:
         #set the font properties for row 1
         fstyle_bold = sheet_cell.font.bold
         fstyle_italic = sheet_cell.font.italic
         fstyle_size = sheet_cell.font.size
         #print("I am setting", fstyle_bold, "font properties of", row, col)
         font_prpts= { "name": "Arial",
              "size": fstyle_size,
              "bold": fstyle_bold,
              "italic": fstyle_italic,
              "vertAlign":"baseline",
              "underline":"none",
              "strike":False }
         font = Font(**font_prpts)
         sheet_cell.font = font
       elif Font_Size_Switch and index != 1:
         fstyle_bold = sheet_cell.font.bold
         fstyle_italic = sheet_cell.font.italic
         fstyle_size = font_size
        #print("I am setting", fstyle_bold, "font properties of", row, col)
        #set the font properties for remaining rows
         font_prpts= { "name": "Arial",
              "size": fstyle_size if Font_Size_Switch else 9.0,
              "bold": fstyle_bold,
              "italic": fstyle_italic,
              "vertAlign":"baseline",
              "underline":"none",
              "strike":False }
         font = Font(**font_prpts)
         sheet_cell.font = font
#*********This block sets the formatting properties*****************#
       if Border_Switch == True and not 'PPE' in ws.title:
          if sheet_cell.border.left.border_style == "thin" or sheet_cell.border.right.border_style == "thin":
             sheet_cell.border.left.border_style = 'none'
             sheet_cell.border.right.border_style = 'none'
             print("a border is removed at cell location", col, ws.title)
       if row == start_row and sheet_cell.value=="Notes" or sheet_cell.value=="Note":
             sheet_cell.alignment = Alignment(horizontal='center',
                                              vertical = 'bottom',
                                              wrap_text=True)
       if row == start_row and sheet_cell.value=="Consolidated":
             sheet_cell.alignment = Alignment(horizontal='left',
                                              vertical = 'bottom',
                                              wrap_text=False)
       elif row == start_row and sheet_cell.value!="Notes" or sheet_cell.value=="Note":
               sheet_cell.alignment = Alignment(horizontal='right',
                                                vertical = 'bottom',
                                                wrap_text=True)
       elif row != start_row and col == start_col:
             cell_index = sheet_cell.alignment.indent
             sheet_cell.alignment = Alignment(horizontal='left',
                                                vertical = 'bottom',
                                                wrap_text=True,
                                                indent=cell_index)
       elif col==col_index and notes_exists:
               sheet_cell.alignment = Alignment(horizontal='right',
                                                vertical = 'bottom',
                                                wrap_text=True)
               sheet_cell.number_format = 'General'

       else:
               sheet_cell.alignment = Alignment(horizontal='right',
                                                vertical = 'bottom',
                                                wrap_text=False)

    for irow in indx_sheet.iter_rows(min_row=2, max_row=indx_sheet.max_row, min_col=1, max_col=indx_sheet.max_column):
     for icell in irow:
      if icell.value==ws.title:
        indx_sheet.cell(row=icell.row, column=add_col_idx, value=num_count)

wb.save('DoT (Cluster) AR_Offline_Generated.xlsx')

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Start of Year
Start of Index
Start of Parameters
Start of Lineage
Start of AR_001_SCI
Merged_range set()
Notes Index position 3
I am column 2 and I am in condition 2
Condition 4 start 4
I am column 4 and I am in condition 4 4 7 [4, 5, 6, 7]
I am columns 5 I am in condition 0
I am columns 6 I am in condition 0
I am columns 7 I am in condition 0
Numeric columns 5 AR_001_SCI
2 26.09 for label
26.09
I am using 7.82 for Notes columns in sheet AR_001_SCI
I am using 9.91 for amount
I am using 9.91 for amount
I am using 9.91 for amount
I am using 9.91 for amount
Start of AR_002a_SFP_DoT
Merged_range set()
Notes Index position 3
I am column 2 and I am in condition 2
Condition 4 start 4
I am column 4 and I am in condition 4 4 5 [4, 5]
I am columns 5 I am in condition 0
Condition 4 start 7
I am column 7 and I am in condition 4 7 9 [7, 8, 9]
I am columns 8 I am in condition 0
Numeric columns 5 AR_002a_SFP_DoT
2 26.09 for label
26.09
I am using 7.82 for Notes columns in sheet AR_002a_SFP_DoT
I am u

In [ ]:
column_widths = [8,30,32,8,8,8]
def adjust_columns(line, column_widths):
  columns = line.split(',')

  adjusted_columns = []
  for i, col in enumerate(columns):
      if len(col) < column_widths[i]:
        print(col)
        adjusted_columns.append(col.ljust(column_widths[i]))
      else:
        adjusted_columns.append(col)
  return ' '.join(adjusted_columns)

def process_files(input_file, outputfile, column_widths):
  with open(input_file, 'r') as file:
   lines = file.readlines()
  with open(outputfile, 'w') as file:
    for line in lines:
        adjusted_line = adjust_columns(line, column_widths)
        file.write(adjusted_line + '\n')
input_file = 'Initiative to Program.txt'
outputfile = 'Initiative to Program_adjusted.txt'
column_widths = [8,30,32,1,8,8,8,1]
process_files(input_file, outputfile, column_widths)


0000001
NODENAME
PPM_HIER
0000002
0000002
PORTFOLIO_GUID
0000003
0000003
PROGRAM
10004
0000004
0000004
SUB_PROGRAM
30010
0000005
0000005
INITIATIVE_GUID
0000000
0000006
INITIATIVE_GUID
0000000
0000007
INITIATIVE_GUID
0000000
0000008
INITIATIVE_GUID
0000000
0000009
INITIATIVE_GUID
0000000
0000010
INITIATIVE_GUID
0000000
0000011
INITIATIVE_GUID
0000000
0000012
INITIATIVE_GUID
0000000
0000013
INITIATIVE_GUID
0000000
0000014
INITIATIVE_GUID
0000000
0000015
INITIATIVE_GUID
0000000
0000016
INITIATIVE_GUID
0000000
0000017
INITIATIVE_GUID
0000000
0000018
INITIATIVE_GUID
0000000
0000019
INITIATIVE_GUID
0000000
0000020
INITIATIVE_GUID
0000000
0000021
INITIATIVE_GUID
0000000
0000022
INITIATIVE_GUID
0000000
0000023
INITIATIVE_GUID
0000000
0000024
INITIATIVE_GUID
0000000
0000025
SUB_PROGRAM
30089
0000026
0000026
INITIATIVE_GUID
0000000
0000027
INITIATIVE_GUID
0000000
0000028
INITIATIVE_GUID
0000000
0000029
SUB_PROGRAM
30001
0000030
0000030
INITIATIVE_GUID
0000000
0000031
SUB_PROGRAM
30092
0000032
0

In [ ]:
# This is to find out how many invoices are created for each of the PO Line item and wether the value of invoice match the PO line item
import pandas as pd
from google.colab import data_table
from IPython.display import display
import matplotlib.pyplot as plt

base_df = pd.read_excel('/content/POLineItemData01To032024.XLSX')
invoice_df = pd.read_excel('/content/POInvoiceData01to032024.XLSX')
doctext = pd.read_excel('/content/DocumentTypeText.XLSX')
numeric_columns = ['0AMOUNT']

#base_df = base_df.drop_duplicates(subset=['Company code', 'Purchasing Document', 'Purchasing Document Item', 'Schedule line number'], keep='last').reset_index()

po_grouped = base_df.groupby(['Company code', 'Purchasing Document', 'Purchasing Document Item', 'Purchasing document type','Account Assignment Category'])['0AMOUNT'].agg(['sum', 'count']).reset_index()
po_grouped = pd.merge(po_grouped , doctext, on='Purchasing document type', how='inner')
po_grouped = po_grouped.rename(columns={'count': 'PO_LI_CNT'})

inv_grouped = invoice_df.groupby(['Company code', 'Purchasing Document', 'Purchasing Document Item'])['0AMOUNT'].agg(['sum', 'count']).reset_index()
inv_grouped = inv_grouped.rename(columns={'count': 'INV_LI_CNT'})

merged_po_inv = pd.merge(po_grouped, inv_grouped, on=['Company code', 'Purchasing Document', 'Purchasing Document Item'], how = 'outer')
merged_po_inv = merged_po_inv.rename(columns={'sum_x': 'PO_SUM', 'sum_y': 'INV_SUM'})
merged_po_inv = merged_po_inv[['Company code','Purchasing Document', 'Purchasing Document Item', 'Purchasing document type','Long Description','Account Assignment Category','PO_SUM', 'PO_LI_CNT', 'INV_SUM', 'INV_LI_CNT']]


base_filtered_pos = base_df.groupby(['Purchasing document type', 'Account Assignment Category'])['0AMOUNT'].agg(['sum', 'count']).reset_index()
base_filtered_pos = base_filtered_pos.rename(columns={'count': 'Total Line Items', 'sum': 'Total Amount'})

filtered_po_inv = merged_po_inv[merged_po_inv["PO_LI_CNT"] < merged_po_inv["INV_LI_CNT"]]
filtered_po_inv_cnt = filtered_po_inv.groupby(['Purchasing document type', 'Account Assignment Category'])['PO_LI_CNT'].agg(['sum']).reset_index()
filtered_po_inv_cnt = filtered_po_inv_cnt.rename(columns={'sum': 'Inv Lines GT PO Lines'})

filtered_po_inv_amt = filtered_po_inv.groupby(['Purchasing document type', 'Account Assignment Category'])['PO_SUM'].agg(['sum']).reset_index()
filtered_po_inv_amt = filtered_po_inv_amt.rename(columns={'sum':'Sum of InvLinesGTPoLines'})



hist_df = pd.merge(base_filtered_pos, filtered_po_inv_cnt, on=['Purchasing document type','Account Assignment Category'], how='left')
hist_df = pd.merge(hist_df, filtered_po_inv_amt, on=['Purchasing document type','Account Assignment Category'], how='left')
hist_df = hist_df.T.drop_duplicates().T
hist_df = hist_df[['Purchasing document type', 'Account Assignment Category', 'Total Amount', 'Total Line Items', 'Inv Lines GT PO Lines', 'Sum of InvLinesGTPoLines']]

hist_df = pd.merge(hist_df , doctext, on='Purchasing document type', how='inner')
hist_df = hist_df[['Purchasing document type', 'Long Description', 'Account Assignment Category','Total Line Items', 'Total Amount', 'Inv Lines GT PO Lines', 'Sum of InvLinesGTPoLines']]
hist_df['Total Amount'] = hist_df['Total Amount'].astype(int)
#hist_df['Sum of InvLinesGTPoLines'] = hist_df['Sum of InvLinesGTPoLines'].astype(int)
hist_df['% diff'] = (hist_df['Sum of InvLinesGTPoLines'] / hist_df['Total Amount']) * 100

Subtotals = hist_df.groupby(['Purchasing document type']).sum().reset_index()
Subtotals['Long Description'] = 'Subtotal'
Subtotals['Account Assignment Category'] = ' '
Subtotals['% diff'] = (Subtotals['Sum of InvLinesGTPoLines'] / Subtotals['Total Amount']) * 100

totals = pd.DataFrame({'Purchasing document type':['Total'], 'Long Description':[''],'Account Assignment Category':[''],'Total Line Items':[hist_df['Total Line Items'].sum()],'Total Amount':[hist_df['Total Amount'].sum()], 'Inv Lines GT PO Lines':[hist_df['Inv Lines GT PO Lines'].sum()], 'Sum of InvLinesGTPoLines':[hist_df['Sum of InvLinesGTPoLines'].sum()]})
totals['% diff'] = (totals['Sum of InvLinesGTPoLines'] / totals['Total Amount']) * 100
hist_df = pd.concat([hist_df, Subtotals],ignore_index = True)
hist_df = hist_df.sort_values(by=['Purchasing document type', 'Account Assignment Category'], ascending=[True, False])
hist_df = pd.concat([hist_df, totals],ignore_index = True)

#hist_df['% diff'] = hist_df['% diff'].astype(int)
pd.set_option("colheader_justify", "left")
pd.set_option("display.precision", 0)
#ax = merged_po_inv.plot.hexbin(x='Purchasing Document Item', y='INV_LI_CNT', gridsize=20)

format_columns = ["Total Amount","Sum of InvLinesGTPoLines"]
hist_df[format_columns] = hist_df[format_columns].map(lambda x: "${:,.2f}".format(x))

display(hist_df)
#styled_df = hist_df.style.set_properties(subset=['Total Amount','Sum of InvLinesGTPoLines' ], **{'text-align': 'right'}).set_properties(subset=['Purchasing document type'], **{'text-align': 'left'})
data_table.DataTable(hist_df, include_index=False)


#print(styled_df)
#display(styled_df)

,Purchasing document type,Long Description,Account Assignment Category,Total Line Items,Total Amount,Inv Lines GT PO Lines,Sum of InvLinesGTPoLines,% diff
0,ECPO,SRM Purchase Order,U,1,"$77,120.00",1,"$77,120.00",1
1,ECPO,SRM Purchase Order,P,6408,"$322,491,082.00",906,"$273,538,843.88",85
2,ECPO,SRM Purchase Order,K,26958,"$107,435,225.00",847,"$83,469,874.11",78
3,ECPO,SRM Purchase Order,F,1624,"$1,987,333.00",126,"$130,448.23",7
4,ECPO,Subtotal,,34991,"$431,990,760.00",188,"$357,216,286.22",83
5,NB,Standard PO,P,1408,"$1,602,361,894.00",854,"$1,379,675,975.69",86
6,NB,Standard PO,K,113,"$138,766,875.00",7,"$133,422,613.92",96
7,NB,Standard PO,F,5639,"$212,090,127.00",2769,"$165,954,164.48",78
8,NB,Standard PO,BL,2117,"$46,800,319.00",NaN,$nan,NaN
9,NB,Subtotal,,9277,"$2,000,019,215.00",3693,"$1,679,052,754.09",84


,Purchasing document type,Long Description,Account Assignment Category,Total Line Items,Total Amount,Inv Lines GT PO Lines,Sum of InvLinesGTPoLines,% diff
0,ECPO,SRM Purchase Order,U,1,"$77,120.00",1,"$77,120.00",1
1,ECPO,SRM Purchase Order,P,6408,"$322,491,082.00",906,"$273,538,843.88",85
2,ECPO,SRM Purchase Order,K,26958,"$107,435,225.00",847,"$83,469,874.11",78
3,ECPO,SRM Purchase Order,F,1624,"$1,987,333.00",126,"$130,448.23",7
4,ECPO,Subtotal,,34991,"$431,990,760.00",188,"$357,216,286.22",83
5,NB,Standard PO,P,1408,"$1,602,361,894.00",854,"$1,379,675,975.69",86
6,NB,Standard PO,K,113,"$138,766,875.00",7,"$133,422,613.92",96
7,NB,Standard PO,F,5639,"$212,090,127.00",2769,"$165,954,164.48",78
8,NB,Standard PO,BL,2117,"$46,800,319.00",NaN,$nan,NaN
9,NB,Subtotal,,9277,"$2,000,019,215.00",3693,"$1,679,052,754.09",84


In [ ]:
# This is a copy for testing
import pandas as pd
from google.colab import data_table
from IPython.display import display
import matplotlib.pyplot as plt

pd.set_option("colheader_justify", "left")
pd.set_option("display.precision", 0)
base_df = pd.read_excel('/content/POSample.xlsx')
invoice_df = pd.read_excel('/content/InvoiceSample.xlsx')
doctext = pd.read_excel('/content/DocumentTypeText.XLSX')
numeric_columns = ['0AMOUNT']

base_df = base_df.drop_duplicates(subset=['Company code', 'Purchasing Document', 'Purchasing Document Item', 'Schedule line number'], keep='last').reset_index()

po_grouped = base_df.groupby(['Company code', 'Purchasing Document', 'Purchasing Document Item', 'Purchasing document type'])['0AMOUNT'].agg(['sum', 'count']).reset_index()
po_grouped = pd.merge(po_grouped , doctext, on='Purchasing document type', how='inner')
po_grouped = po_grouped.rename(columns={'count': 'PO_LI_CNT'})

inv_grouped = invoice_df.groupby(['Company code', 'Purchasing Document', 'Purchasing Document Item'])['0AMOUNT'].agg(['sum', 'count']).reset_index()
inv_grouped = inv_grouped.rename(columns={'count': 'INV_LI_CNT'})

merged_po_inv = pd.merge(po_grouped, inv_grouped, on=['Company code', 'Purchasing Document', 'Purchasing Document Item'], how = 'outer')
merged_po_inv = merged_po_inv.rename(columns={'sum_x': 'PO_SUM', 'sum_y': 'INV_SUM'})
merged_po_inv = merged_po_inv[['Company code','Purchasing Document', 'Purchasing Document Item', 'Purchasing document type','Long Description','PO_SUM', 'PO_LI_CNT', 'INV_SUM', 'INV_LI_CNT']]

base_filtered_pos = base_df.groupby(['Purchasing document type'])['0AMOUNT'].agg(['sum', 'count']).reset_index()
base_filtered_pos = base_filtered_pos.rename(columns={'count': 'Total Line Items', 'sum': 'Total Amount'})
filtered_po_inv = merged_po_inv[merged_po_inv["PO_LI_CNT"] < merged_po_inv["INV_LI_CNT"]]
filtered_po_inv_cnt = filtered_po_inv.groupby(['Purchasing document type'])['PO_LI_CNT'].agg(['sum']).reset_index()
filtered_po_inv_cnt = filtered_po_inv_cnt.rename(columns={'sum': 'Inv Lines GT PO Lines'})
filtered_po_inv_amt = filtered_po_inv.groupby(['Purchasing document type'])['PO_SUM'].agg(['sum']).reset_index()
filtered_po_inv_amt = filtered_po_inv_amt.rename(columns={'sum':'Sum of InvLinesGTPoLines'})


hist_df = pd.concat([base_filtered_pos, filtered_po_inv_cnt, filtered_po_inv_amt], axis=1)
hist_df = hist_df.T.drop_duplicates().T
hist_df['Total Amount'] = hist_df['Total Amount'].astype(int)
hist_df['Sum of InvLinesGTPoLines'] = hist_df['Sum of InvLinesGTPoLines'].astype(int)

summary_styler = hist_df.agg(["sum"]).style.format(precision=3).relabel_index(["Sum"])
hist_df.style.format(precision=1).concat(summary_styler)
#ax = merged_po_inv.plot.hexbin(x='Purchasing Document Item', y='INV_LI_CNT', gridsize=20)

format_columns = ["Total Amount","Sum of InvLinesGTPoLines"]
hist_df[format_columns] = hist_df[format_columns].map(lambda x: "${:,.2f}".format(x))
#styled_df = hist_df.style.set_properties(subset=['Total Amount','Sum of InvLinesGTPoLines' ], **{'text-align': 'right'}).set_properties(subset=['Purchasing document type'], **{'text-align': 'left'})
data_table.DataTable(hist_df, include_index=False)
display(hist_df)
#print(styled_df)
#display(styled_df)

,Purchasing document type,Total Amount,Total Line Items,Inv Lines GT PO Lines,Sum of InvLinesGTPoLines
0,NB,"$19,400.00",3,1,"$4,700.00"


In [ ]:
# This is to find POs associated with WBSes
import pandas as pd
import seaborn as sns
from IPython.display import display

base_df = pd.read_excel('/content/POScheduleLineData01To032025.XLSX')
doctext = pd.read_excel('/content/DocumentTypeText.XLSX')
wbstoprj_map =  pd.read_excel('/content/WBStoProject.XLSX')
proj_text = pd.read_excel('/content/Project Text.XLSX')

base_df = base_df[(base_df['Work Breakdown Structure'].notna()) & (base_df['Work Breakdown Structure'] != '0')]

merged_df = pd.merge(base_df, wbstoprj_map, left_on='Work Breakdown Structure', right_on='WBS Element', how='left', suffixes=('_left', '_right'))
merged_df = pd.merge(merged_df, proj_text, on='Project Definition', how='left')

merged_df = merged_df.groupby(['Company code', 'Purchasing document type'])['0AMOUNT'].agg(['sum', 'count'])
merged_df = merged_df.style.format(precision=3, thousands=",", decimal=".", na_rep="", prefix="$")

display(merged_df)




KeyError: ''

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display
from google.colab import data_table
import plotly.express as px
from google.colab import data_table

base_df = pd.read_csv('/content/POScheduleLineData01To032024.csv')
#invoice_df = pd.read_excel('/content/POInvoiceData01to032025.XLSX')
doctext = pd.read_excel('/content/DocumentTypeText.XLSX')

base_df = base_df.loc[base_df.groupby(['Purchase Document', 'Purchase Document Item', 'Schedule line number' ])['0AMOUNT'].idxmax()]
base_df = base_df.reset_index()

test = base_df[base_df['Purchasing document type'] == 'ECPO']
test =test[test['Account Assignment Category'] == 'P']


base_df = base_df.rename(columns={'Purchasing document type': 'Doc Type', 'Planned delivery date of document schedu':'Delivery Date'})
base_df = base_df.reset_index()
grouped = base_df.groupby(['Company code', 'Doc Type','Account Assignment Category'])['0AMOUNT'].agg(['sum', 'count']).reset_index()
base_df['Document Date'] = pd.to_datetime(base_df['Document Date'], format='%d/%m/%Y')
base_df['Delivery Date'] = pd.to_datetime(base_df['Delivery Date'], format='%d/%m/%Y')
#condition = base_df.loc[:, 'Work Breakdown Structure'].notna() & base_df.loc[:, 'Work Breakdown Structure'] != '00000000'

#***Doc Date EQUAL Delivery Date Filter********
filtered_df_a = base_df[base_df['Document Date'] == base_df['Delivery Date']]
filtered_df_a = filtered_df_a.groupby(['Company code', 'Doc Type', 'Account Assignment Category'])['0AMOUNT'].agg(['sum', 'count']).reset_index()
#filtered_group[sum] = grouped[sum].apply(lambda x: "${:,.2f}".format(x))
filtered_group_EQ = filtered_df_a.rename(columns={'sum': 'Total Amount_EQ', 'count': 'DocDate = DelDate'})


#***Doc Date Less Than Delivery Date Filter********
filtered_df_b = base_df[base_df['Document Date'] < base_df['Delivery Date']]
filtered_df_b = filtered_df_b.groupby(['Company code', 'Doc Type', 'Account Assignment Category'])['0AMOUNT'].agg(['sum', 'count']).reset_index()
filtered_group_LT = filtered_df_b.rename(columns={'sum': 'Total Amount_LT', 'count': 'DocDate LT DelDate'})
#***Doc Date Greater Than Delivery Date Filter********
filtered_df_c = base_df[base_df['Document Date'] > base_df['Delivery Date']]
filtered_df_c = filtered_df_c.groupby(['Company code', 'Doc Type','Account Assignment Category'])['0AMOUNT'].agg(['sum', 'count']).reset_index()
filtered_group_GT = filtered_df_c.rename(columns={'sum': 'Total Amount_GT', 'count': 'DocDate GT DelDate'})


grouped = grouped.rename(columns={'sum': 'Total Amount', 'count': 'Total_Count'})
grouped = grouped.reset_index()
#display(grouped)
merged_df = pd.merge(grouped, doctext, left_on='Doc Type', right_on='Purchasing document type', how='left')
merged_df = merged_df.rename(columns={'Long Description': 'Doc Type Desc'})
merged_df = merged_df[['Company code', 'Doc Type', 'Doc Type Desc', 'Account Assignment Category','Total Amount', 'Total_Count']]

merged_df = pd.merge(merged_df, filtered_group_EQ, on=['Company code', 'Doc Type', 'Account Assignment Category'], how='outer')
merged_df = pd.merge(merged_df, filtered_group_LT, on=['Company code', 'Doc Type', 'Account Assignment Category'], how='outer')
merged_df = pd.merge(merged_df, filtered_group_GT, on=['Company code', 'Doc Type', 'Account Assignment Category'], how='outer')
merged_df['% DelDateEQDocDate'] =  (merged_df['DocDate = DelDate'] / merged_df['Total_Count']) * 100
merged_df['% DelDateLTDocDate'] =  (merged_df['DocDate LT DelDate'] / merged_df['Total_Count']) * 100
merged_df['% DelDateGTDocDate'] =  (merged_df['DocDate GT DelDate'] / merged_df['Total_Count']) * 100
merged_df['Total_Count_log'] = np.log1p(merged_df['Total_Count'])
merged_df['Total Amount_log'] = np.log1p(merged_df['Total Amount'])

fig = px.sunburst(merged_df, path=['Company code', 'Doc Type Desc', 'Account Assignment Category'], values='Total_Count_log' , color='Total Amount_log', hover_data=['Total Amount_EQ', 'Total Amount_LT', 'Total Amount_GT','DocDate = DelDate', 'DocDate LT DelDate', 'DocDate GT DelDate'])
fig.show()

data_table.enable_dataframe_formatter()
data_table.DataTable(filtered_group_EQ, include_index=False, min_width=100)

#***Format the Amount********
format_columns = ['Total Amount_GT','Total Amount_LT','Total Amount_EQ','Total Amount']
merged_df[format_columns] = merged_df[format_columns].map(lambda x: "${:,.2f}".format(x))
#print(filtered_group_GT)
#filtered_group_GT['Total Amount_LT'] = filtered_group_GT['Total Amount_LT'].apply(lambda x: "${:,.2f}".format(x))
#filtered_group_GT['Total Amount_EQ'] = filtered_group_GT['Total Amount_LT'].apply(lambda x: "${:,.2f}".format(x))
#styled_df = merged_df.style.format({'DocDate = DelDate': "{:.0f}", 'DocDate LT DelDate': "{:.0f}", 'DocDate GT DelDate': "{:.0f}"}).set_properties(subset=['Doc Type', 'Doc Type Desc', 'Account Assignment Category'], **{'text-align': 'left'}).set_properties(subset=['Total Amount_GT', 'Total Amount_LT', 'Total Amount_EQ', 'Total Amount'], **{'text-align': 'right'})


#print(styled_df)
#display(styled_df
data_table.DataTable(merged_df, include_index=False, min_width=100)


,Company code,Doc Type,Doc Type Desc,Account Assignment Category,Total Amount,Total_Count,Total Amount_EQ,DocDate = DelDate,Total Amount_LT,DocDate LT DelDate,Total Amount_GT,DocDate GT DelDate,% DelDateEQDocDate,% DelDateLTDocDate,% DelDateGTDocDate,Total_Count_log,Total Amount_log
0,1000,ECPO,SRM Purchase Order,K,"$78,633,797.14",5762,"$33,801,879.98",1354,"$12,811,687.18",2851,"$32,020,229.98",1557,23,49,3e+01,9e+00,18
1,1000,ECPO,SRM Purchase Order,P,"$197,245,355.28",2400,"$100,987,553.36",846,"$46,099,874.16",894,"$50,157,927.76",660,35,37,3e+01,8e+00,19
2,1000,ECPO,SRM Purchase Order,U,"$154,240.00",1,"$154,240.00",1,$nan,NaN,$nan,NaN,100,NaN,NaN,7e-01,12
3,1000,NB,Standard PO,K,"$18,183,589.19",58,"$12,553,564.32",30,"$4,954,634.87",25,"$675,390.00",3,52,43,5e+00,4e+00,17
4,1000,NB,Standard PO,P,"$2,066,207,970.41",1220,"$197,388,323.68",129,"$1,862,435,948.07",1079,"$6,383,698.66",12,11,88,1e+00,7e+00,21
5,1000,ZIC,Intercompany PO,K,"$34,959,068.30",10,"$34,716,953.14",7,"$242,115.16",3,$nan,NaN,70,30,NaN,2e+00,17
6,1000,ZIC,Intercompany PO,P,"$21,121,160,897.73",79,"$12,030,406,595.34",39,"$9,090,754,302.39",40,$nan,NaN,49,51,NaN,4e+00,24
7,1040,ECPO,SRM Purchase Order,K,"$311,964.78",20,"$182,611.96",9,"$129,192.18",9,$160.64,2,45,45,1e+01,3e+00,13
8,1040,ZIC,Intercompany PO,K,"$35,200.00",1,"$35,200.00",1,$nan,NaN,$nan,NaN,100,NaN,NaN,7e-01,10
9,2000,ZIC,Intercompany PO,K,"$2,357,966.82",6,$nan,NaN,$nan,NaN,"$2,357,966.82",6,NaN,NaN,1e+02,2e+00,15


Following Code is to Print PO Documents

In [ ]:
pip install pivottablejs

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display
from google.colab import data_table
import plotly.express as px
from google.colab import data_table


base_df = pd.read_csv('/content/POScheduleLineData01To032024.csv')
#invoice_df = pd.read_excel('/content/POInvoiceData01to032025.XLSX')
doctext = pd.read_excel('/content/DocumentTypeText.XLSX')

base_df = base_df.loc[base_df.groupby(['Purchase Document', 'Purchase Document Item', 'Schedule line number' ])['0AMOUNT'].idxmax()]
base_df = base_df.reset_index()

test = base_df[base_df['Purchasing document type'] == 'ECPO']
test =test[test['Account Assignment Category'] == 'P']


base_df = base_df.rename(columns={'Purchasing document type': 'Doc Type', 'Planned delivery date of document schedu':'Delivery Date'})
base_df = base_df.reset_index()
grouped = base_df.groupby(['Doc Type','Account Assignment Category'])['0AMOUNT'].agg(['sum', 'count']).reset_index()
base_df['Document Date'] = pd.to_datetime(base_df['Document Date'], format='%d/%m/%Y')
base_df['Delivery Date'] = pd.to_datetime(base_df['Delivery Date'], format='%d/%m/%Y')
#condition = base_df.loc[:, 'Work Breakdown Structure'].notna() & base_df.loc[:, 'Work Breakdown Structure'] != '00000000'

#***Doc Date EQUAL Delivery Date Filter********
filtered_df_a = base_df[base_df['Document Date'] == base_df['Delivery Date']]
filtered_df_a = filtered_df_a.groupby(['Doc Type', 'Account Assignment Category'])['0AMOUNT'].agg(['sum', 'count']).reset_index()
#filtered_group[sum] = grouped[sum].apply(lambda x: "${:,.2f}".format(x))
filtered_group_EQ = filtered_df_a.rename(columns={'sum': 'Total Amount_EQ', 'count': 'DocDate = DelDate'})


#***Doc Date Less Than Delivery Date Filter********
filtered_df_b = base_df[base_df['Document Date'] < base_df['Delivery Date']]
filtered_df_b = filtered_df_b.groupby(['Doc Type', 'Account Assignment Category'])['0AMOUNT'].agg(['sum', 'count']).reset_index()
filtered_group_LT = filtered_df_b.rename(columns={'sum': 'Total Amount_LT', 'count': 'DocDate LT DelDate'})
#***Doc Date Greater Than Delivery Date Filter********
filtered_df_c = base_df[base_df['Document Date'] > base_df['Delivery Date']]
filtered_df_c = filtered_df_c.groupby(['Doc Type','Account Assignment Category'])['0AMOUNT'].agg(['sum', 'count']).reset_index()
filtered_group_GT = filtered_df_c.rename(columns={'sum': 'Total Amount_GT', 'count': 'DocDate GT DelDate'})


grouped = grouped.rename(columns={'sum': 'Total Amount', 'count': 'Total_Count'})
grouped = grouped.reset_index()
#display(grouped)
merged_df = pd.merge(grouped, doctext, left_on='Doc Type', right_on='Purchasing document type', how='left')
merged_df = merged_df.rename(columns={'Long Description': 'Doc Type Desc'})
merged_df = merged_df[['Doc Type', 'Doc Type Desc', 'Account Assignment Category','Total Amount', 'Total_Count']]

merged_df = pd.merge(merged_df, filtered_group_EQ, on=['Doc Type', 'Account Assignment Category'], how='outer')
merged_df = pd.merge(merged_df, filtered_group_LT, on=['Doc Type', 'Account Assignment Category'], how='outer')
merged_df = pd.merge(merged_df, filtered_group_GT, on=['Doc Type', 'Account Assignment Category'], how='outer')
merged_df['% DelDateEQDocDate'] =  (merged_df['DocDate = DelDate'] / merged_df['Total_Count']) * 100
merged_df['% DocDateLTDelDate'] =  (merged_df['DocDate LT DelDate'] / merged_df['Total_Count']) * 100
merged_df['% DocDateGTDelDate'] =  (merged_df['DocDate GT DelDate'] / merged_df['Total_Count']) * 100
merged_df['Total_Count_log'] = np.log1p(merged_df['Total_Count'])
merged_df['Total Amount_log'] = np.log1p(merged_df['Total Amount'])

#fig = px.sunburst(merged_df, path=['Doc Type Desc', 'Account Assignment Category'], values='Total_Count_log' , color='Total Amount_log', hover_data=['Total Amount_EQ', 'Total Amount_LT', 'Total Amount_GT','DocDate = DelDate', 'DocDate LT DelDate', 'DocDate GT DelDate'])
#fig.show()

Subtotals = merged_df.groupby(['Doc Type']).sum().reset_index()
Subtotals['Doc Type Desc'] = 'Subtotal'
Subtotals['Account Assignment Category'] = ' '
Subtotals['% DelDateEQDocDate'] =  (Subtotals['DocDate = DelDate'] / Subtotals['Total_Count']) * 100
Subtotals['% DocDateLTDelDate'] =  (Subtotals['DocDate LT DelDate'] / Subtotals['Total_Count']) * 100
Subtotals['% DocDateGTDelDate'] =  (Subtotals['DocDate GT DelDate'] / Subtotals['Total_Count']) * 100

Totals = pd.DataFrame({'Doc Type':['Total'], 'Account Assignment Category':[''],'Total Amount':[merged_df['Total Amount'].sum()],'Total_Count':[merged_df['Total_Count'].sum()],'DocDate = DelDate':[merged_df['DocDate = DelDate'].sum()], 'DocDate GT DelDate':[merged_df['DocDate GT DelDate'].sum()],\
                       'DocDate LT DelDate':[merged_df['DocDate LT DelDate'].sum()], 'Total Amount_EQ':[merged_df['Total Amount_EQ'].sum()],'Total Amount_GT':[merged_df['Total Amount_GT'].sum()],'Total Amount_LT':[merged_df['Total Amount_LT'].sum()]})

Totals['% DelDateEQDocDate'] = (Totals['DocDate = DelDate'] / Totals['Total_Count']) * 100
Totals['% DocDateLTDelDate'] =  (Totals['DocDate LT DelDate'] / Totals['Total_Count']) * 100
Totals['% DocDateGTDelDate'] =  (Totals['DocDate GT DelDate'] / Totals['Total_Count']) * 100
merged_df = pd.concat([merged_df, Subtotals],ignore_index = True)

merged_df = merged_df.sort_values(by=['Doc Type', 'Account Assignment Category'], ascending=[True, False])
merged_df = pd.concat([merged_df, Totals],ignore_index = True)

data_table.enable_dataframe_formatter()
data_table.DataTable(filtered_group_EQ, include_index=False, min_width=100)

#***Format the Amount********
format_columns = ['Total Amount_GT','Total Amount_LT','Total Amount_EQ','Total Amount']
merged_df[format_columns] = merged_df[format_columns].map(lambda x: "${:,.2f}".format(x))
#print(filtered_group_GT)
#filtered_group_GT['Total Amount_LT'] = filtered_group_GT['Total Amount_LT'].apply(lambda x: "${:,.2f}".format(x))
#filtered_group_GT['Total Amount_EQ'] = filtered_group_GT['Total Amount_LT'].apply(lambda x: "${:,.2f}".format(x))
#styled_df = merged_df.style.format({'DocDate = DelDate': "{:.0f}", 'DocDate LT DelDate': "{:.0f}", 'DocDate GT DelDate': "{:.0f}"}).set_properties(subset=['Doc Type', 'Doc Type Desc', 'Account Assignment Category'], **{'text-align': 'left'}).set_properties(subset=['Total Amount_GT', 'Total Amount_LT', 'Total Amount_EQ', 'Total Amount'], **{'text-align': 'right'})


#print(styled_df)
#display(styled_df
data_table.DataTable(merged_df, include_index=False, min_width=100)


,Doc Type,Doc Type Desc,Account Assignment Category,Total Amount,Total_Count,Total Amount_EQ,DocDate = DelDate,Total Amount_LT,DocDate LT DelDate,Total Amount_GT,DocDate GT DelDate,% DelDateEQDocDate,% DocDateLTDelDate,% DocDateGTDelDate,Total_Count_log,Total Amount_log
0,ECPO,SRM Purchase Order,U,"$154,240.00",1,"$154,240.00",1.0,$nan,NaN,$nan,NaN,100.000000,NaN,NaN,0.693147,11.946272
1,ECPO,SRM Purchase Order,P,"$631,320,905.02",6548,"$123,272,080.64",1997.0,"$73,931,607.70",2559.0,"$434,117,216.68",1992.0,30.497862,39.080635,30.421503,8.787068,20.263325
2,ECPO,SRM Purchase Order,K,"$219,913,848.73",26900,"$63,345,985.29",5674.0,"$35,713,991.18",14096.0,"$120,853,872.26",7130.0,21.092937,52.401487,26.505576,10.199919,19.208746
3,ECPO,SRM Purchase Order,F,"$4,151,514.40",1625,"$1,268,100.66",298.0,"$1,185,445.04",976.0,"$1,697,968.70",351.0,18.338462,60.061538,21.600000,7.393878,15.238984
4,ECPO,Subtotal,,"$855,540,508.15",35074,"$188,040,406.59",7970.0,"$110,831,043.92",17631.0,"$556,669,057.64",9473.0,22.723385,50.268005,27.008610,27.074012,66.657327
5,NB,Standard PO,P,"$2,584,331,191.44",1335,"$702,667,911.67",194.0,"$1,874,175,865.57",1124.0,"$7,487,414.20",17.0,14.531835,84.194757,1.273408,7.197435,21.672733
6,NB,Standard PO,K,"$156,695,293.07",108,"$137,595,532.00",58.0,"$18,424,371.07",47.0,"$675,390.00",3.0,53.703704,43.518519,2.777778,4.691348,18.869814
7,NB,Standard PO,F,"$353,536,747.27",5657,"$29,602,023.29",385.0,"$235,503,641.64",3740.0,"$88,431,082.34",1532.0,6.805727,66.112781,27.081492,8.640826,19.683498
8,NB,Subtotal,,"$3,094,563,231.78",7100,"$869,865,466.96",637.0,"$2,128,103,878.28",4911.0,"$96,593,886.54",1552.0,8.971831,69.169014,21.859155,20.529609,60.226044
9,ZBO,Blanket PO,F,"$1,022,160.20",12,"$1,022,160.20",12.0,$nan,NaN,$nan,NaN,100.000000,NaN,NaN,2.564949,13.837430


# New section

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display
from google.colab import data_table
import plotly.express as px
from google.colab import data_table


base_df = pd.read_csv('/content/POScheduleLineData01To032024.csv')
#invoice_df = pd.read_excel('/content/POInvoiceData01to032025.XLSX')
doctext = pd.read_excel('/content/DocumentTypeText.XLSX')
mat_group = pd.read_excel('/content/materialGroup.XLSX')

base_df = base_df.loc[base_df.groupby(['Purchase Document', 'Purchase Document Item', 'Schedule line number' ])['0AMOUNT'].idxmax()]

filtered_df_a = base_df[(base_df['Document Date'] == base_df['Planned delivery date of document schedu']) & (base_df['Account Assignment Category'] =='P')]
filtered_df_b = base_df[(base_df['Document Date'] == base_df['Planned delivery date of document schedu']) & (base_df['Account Assignment Category'] =='P')]
filtered_df_b = filtered_df_b[(filtered_df_b['Material group'].astype(str).str.contains('72131700'))]
filtered_df_b = pd.merge(filtered_df_b , mat_group, on = 'Material group', how='left')

filtered_df_b = filtered_df_b.groupby(['Purchasing Document', 'Material group'])['0AMOUNT'].agg(['sum', 'count']).reset_index()
display(filtered_df_b)
filtered_df_a = filtered_df_a.groupby(['Purchasing document type', 'Material group'])['0AMOUNT'].agg(['sum', 'count']).reset_index()
filtered_df_a = pd.merge(filtered_df_a , doctext, on = 'Purchasing document type', how='inner')
filtered_df_a = pd.merge(filtered_df_a , mat_group, on = 'Material group', how='left')
filtered_df_a = filtered_df_a[['Material group','Short Description', 'sum', 'count']]
#filtered_df_a = filtered_df_a.groupby(['Material group','Short Description'])['sum'].agg(['sum', 'count']).reset_index()
filtered_df_a['sum'] = filtered_df_a['sum'].map(lambda x: "${:,.2f}".format(x))
filtered_df_b['sum'] = filtered_df_b['sum'].map(lambda x: "${:,.2f}".format(x))


data_table.enable_dataframe_formatter()
data_table.DataTable(filtered_df_a, include_index=False, min_width=100)


,Purchasing Document,Material group,sum,count
0,4200306941,72131700,33305306.00,1
1,4200307611,72131700,1341868.02,1
2,4200307631,72131700,14000.00,1
3,4200307850,72131700,63484.00,1
4,4200308401,72131700,23153.20,1
5,4200308977,72131700,8290.00,1
6,4200309213,72131700,71659.28,1
7,4200309215,72131700,3775.30,1
8,4200309291,72131700,378502.00,1
9,4200309369,72131700,28901904.86,1


,Material group,Short Description,sum,count
0,10190000,PEST CONTROL PRODUCT,"$18,200.00",1
1,10191509,DO NOT USE,"$1,005.42",3
2,11110000,EARTH AND STONE,"$21,484.00",1
3,11121600,WOOD,"$7,000.00",1
4,11171500,BASIC STEELS,"$72,190.10",4
...,...,...,...,...
355,86101810,DO NOT USE,"$4,780.54",1
356,93151611,GRANT & CONTRIBUTION,"$11,771,225,426.80",23
357,98084205,OPERATIONS,"$25,451.00",1
358,98084207,CORPORATE SERVICES,"$120,500.00",1


In [ ]:
#**************************************************************************************This is a version that retains the formulas**************************************************************************
from numpy.core.multiarray import concatenate
import pandas as pd
import openpyxl
import numpy as np
import re
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font

wb = openpyxl.load_workbook('/content/DoT (Cluster) AR.xlsx', data_only=False, rich_text=True)
indx_sheet = wb['Index']
add_col_idx = indx_sheet.max_column + 1
#sheet_names = wb.sheetnames
# Use the following SWITCH to define "scope" of the script. False = Do not apply, True = Apply
Font_Size_Switch = True
Row_Height_Switch = True
Border_Switch = True # False will not remove vertical borders
Column_Width_Switch = True
#print(wb.sheetnames)#
for title in wb.sheetnames:
  print("Start of", title)
  ws = wb[title]
#  hidden_flg = ws.sheet_state
  for name_range in wb.defined_names.values():
   char_idx = name_range.attr_text.find('!')
   #print(ws.sheet_state)
   if ws.title==name_range.attr_text[:char_idx] and ws.sheet_state == 'visible':
    #named_array = wb.defined_names[title]
    #print("I am processing",ws.title)
    cell_range=name_range.attr_text
    #print(cell_range)
    start_cell, end_cell = cell_range.split(':')
    sheet, start_cell = start_cell.split('!')
    start_cell = start_cell.replace("$","")
    end_cell = end_cell.replace("$","")
    start_row, start_col = openpyxl.utils.cell.coordinate_to_tuple(start_cell)
    end_row, end_col= openpyxl.utils.cell.coordinate_to_tuple(end_cell)
    row_height = 14.5
    index = 0
    num_count = 0
    cnt_nts = 0
    hidden_columns = []
    notes_exists = False
    min=0
    max=0
    h_min=0
    h_max=0
    true_end = end_col
    hiddenList=[]
    groupedList=[]

    #Thi block loops on rows defined in the named range to find Notes colum
    for row in ws.iter_rows():
        for row_cell in row:
         if row_cell.value =="Notes" or row_cell.value =="Note":
          notes_exists=True
          cnt_nts = row_cell.col_idx
          print("Notes Index position",cnt_nts)
          break
    # This block determines number of numeric columns

    for col_h in range(start_col , end_col + 1):
        sheet_cell = ws.cell(row=end_row, column=col_h)
        #print("sheet scope", sheet_cell.value, col_h, bool )
        #print(col_h, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden, ws.title )
        if col_h in groupedList: #col_h == true_end and not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min:
          ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min = col_h
          ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max = col_h
          num_count += 1
          print("I am columns", col_h,"I am in condition 0")
          continue
        if col_h in hiddenList: #min and max and col_h > min and col_h <= max and not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden :
             print("I am column", col_h, "and I am in condition 1")
             continue
        elif not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min == ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max: #This is a default scenario for visible columns
             #print(type(sheet_cell.value))
             if (re.match(r'^#',last_row_values.number_format)) and col_h != cnt_nts: # isinstance(sheet_cell.value, (int, float)): #
                  num_count += 1
                  print("I am column", col_h, "and I am in condition 2")
        elif ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min != ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max:   #This capture the first cell of hidden cell range
              hiddenList.clear()
              for i in range(ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max + 1):
                            hiddenList.append(i)
              h_min = ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min
              h_max = ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max
              print("I am column", col_h, "and I am in condition 3", hiddenList )
              continue
        elif not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min != ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max:   #This captures any grouped columns that have min & max defined None and are visible in excel
              groupedList.clear()
              for i in range(ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max + 1):
                            groupedList.append(i)
              min = ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min
              max = ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max
              num_count += 1
              print("I am column", col_h, "and I am in condition 4",min,max, groupedList)

    print("Numeric columns", num_count, ws.title)


    #Loop on rows defined in the named range
    for row in range(start_row, end_row + 1):
      index += 1
      if Row_Height_Switch is True:
        #set the row height to 14.5
        ws.row_dimensions[row].height=row_height
        Is_hidden = ws.row_dimensions[row].hidden
      groupedList.clear()
      hiddenList.clear()
      col_index=start_col + 1
# Loop on columns starting from the range defined in Named Range
      for col in range(start_col , end_col + 1):
       sheet_cell = ws.cell(row=row, column=col)
       if index==1: #Row 1
        last_row_values = ws.cell(row=end_row, column=col)
#******This block is to hide columns that come as visible due to excel functionality to Group columns that are hidden consecutively hidden********
        if col in groupedList: #col_h == true_end and not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min:
          fstyle_bold = sheet_cell.font.bold
          fstyle_italic = sheet_cell.font.italic
        if col_h in hiddenList: #min and max and col_h > min and col_h <= max and not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden :
          ws.column_dimensions[openpyxl.utils.get_column_letter(col)].hidden = True
          continue
        elif not ws.column_dimensions[openpyxl.utils.get_column_letter(col)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min == ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max: #This is a default scenario for visible columns
          fstyle_bold = sheet_cell.font.bold
          fstyle_italic = sheet_cell.font.italic
        elif ws.column_dimensions[openpyxl.utils.get_column_letter(col)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min != ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max:   #This capture the first cell of hidden cell range
          hiddenList.clear()
          for i in range(ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max + 1):
                            hiddenList.append(i)
          print("I am column", col_h, "and I am in condition 3", hiddenList )
          continue
        elif not ws.column_dimensions[openpyxl.utils.get_column_letter(col)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min != ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max:   #This captures any grouped columns that have min & max defined None and are visible in excel
          groupedList.clear()
          for i in range(ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max + 1):
                            groupedList.append(i)
          fstyle_bold = sheet_cell.font.bold
          fstyle_italic = sheet_cell.font.italic
          print("I am column", col_h, "and I am in condition 4",min,max, groupedList)
#********End of block*************

        if not Font_Size_Switch:
            font_size = sheet_cell.font.size
        else:
            # Set the font SIZE based on numeric columns
            if num_count <= 8:
              font_size = 9
            elif num_count in (9, 10):
              font_size = 8
            else:
              font_size = 7

 #******This block sets the Column Width at row of the defined range as this block is executed within Index 1 if condition above
        if Column_Width_Switch is True:
          if notes_exists and col==col_index:
              if num_count == 1:
                   col_width = 8.91
              elif num_count == 2:
                   col_width = 8.91
              elif num_count==3:
                   col_width = 8.91
              elif num_count==4:
                   col_width=8.18
              elif num_count==5:
                   col_width=7.82
              elif num_count==6:
                   col_width=8.91
              elif num_count==7:
                   col_width=8.55
              elif num_count==8:
                   col_width=8.18
              elif num_count==9:
                   col_width=7.82
              elif num_count==10:
                   col_width=7
              elif num_count==11:
                   col_width=6.18
              else:
                   col_width=6.18
              print("I am using", col_width, "for Notes columns in sheet", ws.title)
                # now update the worksheet with above column widths
              ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=col_width
          if col==start_col and not notes_exists: #This is for column 1
                #Set the default column WIDTH of first colum based on how many numeric columns
                if num_count == 1:
                    col_width = 65.55 + 8.91
                elif num_count == 2:
                    col_width = 53.55 + 8.91
                elif num_count==3:
                    col_width = 41.45 + 8.91
                elif num_count==4:
                    col_width=34.18 + 8.18
                elif num_count==5:
                    col_width=26.09 + 7.82
                elif num_count==6:
                    col_width=49 + 8.91
                elif num_count==7:
                    col_width=40.82 + 8.55
                elif num_count==8:
                    col_width=33.64 + 8.18
                elif num_count==9:
                    col_width=27.55 + 7.82
                elif num_count==10:
                    col_width=27.91 + 7
                elif num_count==11:
                    col_width=61.4 + 6.18
                else:
                    col_width=21.64 + 6.18
                ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=col_width
                # set the column widths in worksheet
                print(col, col_width, "for label")
          if col==start_col and notes_exists: #This is for column 1
                #Set the default column WIDTH of first colum based on how many numeric columns
                if num_count == 1:
                    col_width = 65.55
                elif num_count == 2:
                    col_width = 53.55
                elif num_count==3:
                    col_width = 41.45
                elif num_count==4:
                    col_width=34.18
                elif num_count==5:
                    col_width=26.09
                elif num_count==6:
                    col_width=49
                elif num_count==7:
                    col_width=40.82
                elif num_count==8:
                    col_width=33.64
                elif num_count==9:
                    col_width=27.55
                elif num_count==10:
                    col_width=27.91
                elif num_count==11:
                    col_width=61.4
                else:
                    col_width=21.64
                # set the column widths in worksheet
                print(col, col_width, "for label")
                ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=col_width
                print(ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width)
          if (re.match(r'^#',last_row_values.number_format)) and col!=cnt_nts: #isinstance(last_row_values, (int, float)): #
                #Set the default column WIDTH for numeric column based on how many amount columns are in sheet
                if num_count <= 3 or num_count==6:
                   col_width = 11.45
                elif num_count == 4:
                   col_width = 10.36
                elif num_count==5:
                   col_width = 9.91
                elif num_count==7:
                   col_width=10.91
                elif num_count==8:
                   col_width=10.36
                elif num_count==9:
                   col_width=9.64
                elif num_count==10:
                   col_width=8.82
                else:
                   col_width=7.91
                print("I am using", col_width, "for amount")
                # set the column widths in worksheet
                ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=col_width
        elif not Column_Width_Switch:
          ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width
#********This block sets the font size and formatting properties of cell***************
       font_prpts = {}
       if Font_Size_Switch and index == 1:
         #set the font properties for row 1
         fstyle_bold = sheet_cell.font.bold
         fstyle_italic = sheet_cell.font.italic
         fstyle_size = sheet_cell.font.size
         #print("I am setting", fstyle_bold, "font properties of", row, col)
         font_prpts= { "name": "Arial",
              "size": fstyle_size,
              "bold": fstyle_bold,
              "italic": fstyle_italic,
              "vertAlign":"baseline",
              "underline":"none",
              "strike":False }
         font = Font(**font_prpts)
         sheet_cell.font = font
       elif Font_Size_Switch and index != 1:
         fstyle_bold = sheet_cell.font.bold
         fstyle_italic = sheet_cell.font.italic
         fstyle_size = sheet_cell.font.size
         #print("I am setting", fstyle_bold, "font properties of", row, col)
        #set the font properties for remaining rows
         font_prpts= { "name": "Arial",
              "size": fstyle_size if not Font_Size_Switch else 9.0,
              "bold": fstyle_bold,
              "italic": fstyle_italic,
              "vertAlign":"baseline",
              "underline":"none",
              "strike":False }
       font = Font(**font_prpts)
       sheet_cell.font = font
#*********This block sets the formatting properties*****************#
       if Border_Switch == True and not 'PPE' in ws.title:
          if sheet_cell.border.left.border_style == "thin" or sheet_cell.border.right.border_style == "thin":
             sheet_cell.border.left.border_style = 'none'
             sheet_cell.border.right.border_style = 'none'
             print("a border is removed at cell location", col, ws.title)
       if row == start_row and sheet_cell.value=="Notes" or sheet_cell.value=="Note":
             sheet_cell.alignment = Alignment(horizontal='right',
                                              vertical = 'bottom',
                                              wrap_text=True)
       elif row == start_row and sheet_cell.value!="Notes" or sheet_cell.value=="Note":
               sheet_cell.alignment = Alignment(horizontal='right',
                                                vertical = 'bottom',
                                                wrap_text=True)
       elif row != start_row and col == start_col:
             cell_index = sheet_cell.alignment.indent
             sheet_cell.alignment = Alignment(horizontal='left',
                                                vertical = 'bottom',
                                                wrap_text=True,
                                                indent=cell_index)
       elif col==col_index and notes_exists:
               sheet_cell.alignment = Alignment(horizontal='right',
                                                vertical = 'bottom',
                                                wrap_text=True)
               sheet_cell.number_format = 'General'

       else:
               sheet_cell.alignment = Alignment(horizontal='right',
                                                vertical = 'bottom',
                                                wrap_text=False)

  for irow in indx_sheet.iter_rows(min_row=2, max_row=indx_sheet.max_row, min_col=1, max_col=indx_sheet.max_column):
    for icell in irow:
      if icell.value==ws.title:
        indx_sheet.cell(row=icell.row, column=add_col_idx, value=num_count)

wb.save('test.xlsx')

# **Order APIs**




In [ ]:
from flask import Flask, jsonify, request
from datetime import datetime
import uuid
from flask_ngrok import run_with_ngrok



app = Flask(__name__)
run_with_ngrok(app)
orders_db = {}

users = {
      "test":generate_password_hash("test")
}

@auth.authenticate_password
def authenticate_password(username, password):
    if username in users and check_password_hash(users.get(username), password):
        return username
    return False

# Helper function to generate unique order IDs
def generate_order_id():
    return str(uuid.uuid4())

# Helper function to format date
def get_current_date():
    return datetime.now().strftime('%Y-%m-%d')

# === API Endpoints ===

@app.route('/orders', methods=['POST'])
@auth.login_required
def create_order():
    data = request.get_json()


    # Required fields validation
    if not data.get('customer_id') or not data.get('items') or not data.get('shipping_address'):
        return jsonify({"error": "Missing required fields"}), 400

    # Create a new order with a unique ID
    order_id = generate_order_id()
    order_data = {
        "order_id": order_id,
        "customer_id": data['customer_id'],
        "items": data['items'],
        "shipping_address": data['shipping_address'],
        "payment_method": data.get('payment_method', 'credit_card'),
        "order_date": data.get('order_date', get_current_date()),
        "status": "created"
    }

    # Store the order in the "database"
    orders_db[order_id] = order_data

    return jsonify(order_data), 201

@app.route('/orders/<order_id>', methods=['GET'])
def get_order(order_id):
    """
    Retrieve details of an order.
    """
    order = orders_db.get(order_id)

    if not order:
        return jsonify({"error": "Order not found"}), 404

    return jsonify(order)

@app.route('/orders/<order_id>', methods=['PUT'])
def update_order(order_id):
    """
    Update an existing order.
    """
    order = orders_db.get(order_id)

    if not order:
        return jsonify({"error": "Order not found"}), 404

    data = request.get_json()

    # Only allow updates to items or shipping address (for simplicity)
    if 'items' in data:
        order['items'] = data['items']
    if 'shipping_address' in data:
        order['shipping_address'] = data['shipping_address']
    if 'status' in data:
        order['status'] = data['status']

    return jsonify(order)

@app.route('/orders', methods=['GET'])
def search_orders():
    """
    Search orders based on filter criteria.
    """
    customer_id = request.args.get('customer_id')
    status = request.args.get('status')
    start_date = request.args.get('start_date')
    end_date = request.args.get('end_date')

    filtered_orders = []

    for order in orders_db.values():
        if customer_id and order['customer_id'] != customer_id:
            continue
        if status and order['status'] != status:
            continue
        if start_date and order['order_date'] < start_date:
            continue
        if end_date and order['order_date'] > end_date:
            continue
        filtered_orders.append(order)

    return jsonify(filtered_orders)

@app.route('/orders/<order_id>', methods=['DELETE'])
def delete_order(order_id):
    """
    Delete an existing order.
    """
    if order_id not in orders_db:
        return jsonify({"error": "Order not found"}), 404

    del orders_db[order_id]
    return jsonify({"message": "Order deleted successfully"}), 200

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 199, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 789, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll

In [ ]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 199, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 789, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll